In [3]:
'''Import packages'''
'''Requires numpy, pandas, scikit-learn, and matplotlib/seaborn'''
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Lasso
from scipy.stats import linregress

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")

'''Import script which contains functions'''
import analysis_functions
from analysis_functions import get_r2
from analysis_functions import get_lassoCV
from analysis_functions import perform_randomizedLasso

from IPython.display import display, HTML

#If we want to time the implementation: 
#import time
#start_time = time.time()

/home/prubbens/anaconda3/lib/python3.4/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Import the dataframes: 

In [4]:
'''Import data'''
data_abs = pd.read_csv('data/Chloroplasts_removed/nochloro_absolute_otu.tsv', sep=' ', index_col=None, header=0)
data_rel = pd.read_csv('data/Chloroplasts_removed/nochloro_relative_otu.tsv', sep=' ', index_col=None, header=0)
target = pd.read_csv('data/Chloroplasts_removed/nochloro_HNA_LNA.tsv', sep=' ', index_col=0, header=0)
productivity = pd.read_csv('data/Chloroplasts_removed/productivity_data.tsv', sep=' ', index_col=0, header=0)

In [5]:
'''Set sample names as index and shuffle data'''
data_abs.set_index(target.loc[data_abs.index,'samples'],inplace=True)
data_rel.set_index(target.loc[data_rel.index,'samples'],inplace=True)
target.index = target.samples
data_abs = data_abs.sample(frac=1, random_state=3)
data_rel = data_rel.sample(frac=1, random_state=3)
target = target.sample(frac=1, random_state=3)
productivity = productivity.sample(frac=1, random_state=3)

#Create target columns of HNA-values: 
hna = target.loc[:,'HNA.cells']
hna_rel = hna/target.loc[:,'Total.cells']
hna = pd.Series(hna, index=hna.index)
hna_rel = pd.Series(hna_rel, index=hna.index)

**-- PREPROCESSING OF DATA --**

**1)**: filter out those OTUs which have very low abundances and so give rise to (almost) zero-columns. Therefore an OTU has to have a minimal relative abundance one, defined by the parameter $abun$. 

However, depending on the problem set-up, I use a **second constraint** which states that an OTU must have a relative abundance > $abun$ in one of the **productivity** samples. In this way we're going to bias the OTU-selection towards the ones which are considerably present in the productivity samples. 

In [6]:
'''Filtering based on productivity samples, not needed for first part of analysis'''
#retain only productivity samples 
#productivity = productivity.dropna(subset=['tot_bacprod'])
#remove high productivity samples (>90)
#productivity = productivity[productivity.tot_bacprod < 90]

#idx_prod = productivity.samples.values
#display(idx_prod)
#prod = pd.Series(productivity.tot_bacprod.values, index=idx_prod)
#prod_error = pd.Series(productivity.SD_tot_bacprod.values, index=idx_prod)
#prod_rel_error = prod_error/prod

'Filtering based on productivity samples, not needed for first part of analysis'

In [7]:
'''Parameter abun for initial filtering of OTUs'''
abun = 0.01

In [8]:
from analysis_functions import preprocess_df
data_abs = preprocess_df(data_abs,abun,True)
otus = list(data_abs.columns)

print('Number of OTUs: ' + str(len(otus)))

Number of OTUs: 263


(Note that this number is the same whether we use absolute or relative abundances, as the filtering is based on a minimal _relative_ abundance.)

In [9]:
#Some variables to store information and to create inner and outer CV-folds

cv_out = 10
cv_in = 5
outer_cv = KFold(n_splits=cv_out, shuffle=False)

otu_scores_cv = pd.DataFrame(columns=otus)
r2_cv = np.zeros(cv_out)
thresholds_cv = np.zeros(cv_out)

pred = pd.Series(index=data_abs.index)
final_scores = pd.DataFrame(columns=otus)

thresholds = np.arange(0,1,0.02)
t = 0

Let's first check the performance without using the randomized Lasso: 

In [10]:
from analysis_functions import perform_nested_lasso_cv

alphas_abs, preds_abs = perform_nested_lasso_cv(data_abs[otus], hna, cv_out, cv_in)
alphas_rel, preds_rel = perform_nested_lasso_cv(data_rel[otus], hna, cv_out, cv_in)

r2_abs = get_r2(hna, preds_abs)
r2_rel = get_r2(hna, preds_rel)

print('R² based on absolute abundances: ' + str(r2_abs))
print('R² based on relative abundances: ' + str(r2_rel))

R² based on absolute abundances: 0.886182335215
R² based on relative abundances: 0.787474600975


The $R^2$ is already quite good. Let's check the $R^2$ if we would use the total cell counts to predict the HNA-counts: 

In [11]:
alphas_totcells, preds_totcells = perform_nested_lasso_cv(pd.DataFrame(target['Total.cells']), hna, cv_out, cv_in)
r2_totcells = get_r2(hna, preds_totcells)

print('R² based on total cell counts: ' + str(r2_totcells))

R² based on total cell counts: 0.711185406184


From this we already see that there already is added value in using absolute cell counts to predict the HNA-counts:
- $R^2_{HNA(abs)} = 0.768$
- $R^2_{HNA(Total.cells)} = 0.712$

To do so, we use the **_Randomized Lasso_**: this method makes use of two kinds of randomization in order to select variables (i.e., OTU's) with a certain _stability_: (1) it fits a Lasso to various bootstrap subsamples and (2) it perturbs the initial weighting of certain variables. 

This results in a $score \in [0,1]$ that is assigned to variables, with 0 denoting the case where a variable is never chosen by the Lasso, and 1 denoting the case where a variable always is chosen. In other words, the higher the score, the more important a variable can be considered to be. 

Let's use a **10x5 nested cross-validation** scheme to evaluate our total pipeline: this means that the preprocessing and randomized Lasso is now included in this pipeline (in order to be sure not to overfit, motivated by the paragraph ''7.10.2 The Wrong and Right Way to Do Cross-validation" in ESLII): 

**First goal: ** try to pinpoint those OTU's for which we are sure they are present in the '_HNA-cloud_'. 

In [12]:
for idx_train, idx_test in outer_cv.split(data_abs, hna):
    lassoCV = get_lassoCV(cv_in)
    #scaler = StandardScaler()
    #scaler.fit(data_abs.iloc[idx_train,:])
    #data_abs = pd.DataFrame(scaler.transform(data_abs[otus]),index=data_abs.index,columns=otus)    
    lassoCV.fit(data_abs.iloc[idx_train,:], hna[idx_train])
    mse = np.sum(lassoCV.mse_path_, axis=1)
    mse_min = np.min(mse)
    alpha = lassoCV.alpha_
    
    otu_scores = pd.Series(perform_randomizedLasso(data_abs.iloc[idx_train,:], hna[idx_train], alpha), index=otus)
    otu_scores_cv.loc[t] = otu_scores
    otu_scores.sort_values(ascending=False,inplace=True)
        
    mse_scores = np.zeros(len(thresholds))
    dummy=0
        
    scores = otu_scores
    
    for thr in thresholds: 
        scores = otu_scores[otu_scores.values > thr]
        features_new = scores.index
        if(len(features_new) > 0): 
            lassoCV = get_lassoCV(cv_in)
            lassoCV.fit(data_abs.ix[idx_train,features_new],hna[idx_train])
            #alphas, preds = perform_nested_ridge_cv(data_abs[features_new],hna) #We could use this if we want a different evaluation model
            mse = np.sum(lassoCV.mse_path_, axis=1)
            mse_scores[dummy] = np.min(mse)
        dummy+=1
        
    mse_scores = mse_scores[np.nonzero(mse_scores)]
    #mse_min = mse_scores.min()
    mse_min_idx = mse_scores.argmin()
    thresh_max = thresholds[mse_min_idx]
    thresholds_cv[t] = thresh_max
    optimal_scores = otu_scores[otu_scores.values>thresh_max]
    selected_otus = optimal_scores.index
    
    lassoCV = get_lassoCV(cv_in)
    lassoCV.fit(data_abs.ix[idx_train, selected_otus], hna[idx_train])
    alpha = lassoCV.alpha_
    lasso = Lasso(alpha,max_iter=20000,normalize=True)
    pred_cv_final = cross_val_predict(lasso, data_abs.ix[idx_train, selected_otus], hna[idx_train], cv=cv_in)
    r2_cv[t] = get_r2(pred_cv_final, hna[idx_train])
    pred.iloc[idx_test] = lassoCV.predict(data_abs.ix[idx_test,selected_otus])
    t+=1

In [13]:
r2_final = get_r2(pred,hna)   
print('mean(R²_cv), std(R²_cv): ' + str(r2_cv.mean()) + ', ' +  str(r2_cv.std()))
print('R²_test: ' + str(r2_final))

mean(R²_cv), std(R²_cv): 0.948683335443, 0.0134946321203
R²_test: 0.905700250698


This means that: 
- $R^2_{\text{test}} = 0.906$ with normalization (this becomes 0.86 after standardization (not sure why yet)); 
- $\bar{R^2}_{\text{CV}} = 0.949 \pm 0.013$

This also means that we can now have a look to which extent OTU's are chosen in the various folds according to the randomized Lasso. In other words, we can take their average score, and only look at those OTU's which have a score higher than twice times the standard deviation from the average score (I have to check the following comment, but I think this means that with 95% certainty these OTU's are chosen according to the randomized Lasso); in other words, this means that the ones ranked in the top really are the most important ones: 

In [14]:
mean_otu_scores = otu_scores_cv.mean()
std_otu_scores = otu_scores_cv.std()
mean_otu_scores.sort_values(ascending=False,inplace=True)
avg_thresh = thresholds_cv.mean()
std_thresh = thresholds_cv.std()
otu_scores_cv.to_csv('HNA_scores_OTU_stand_10x5CV_abun0.01.csv')
otus_final = mean_otu_scores[mean_otu_scores > (avg_thresh+2*std_thresh)]
display(otus_final.head(10))

Otu000098    0.693000
Otu000005    0.651333
Otu000025    0.550667
Otu000123    0.544333
Otu000124    0.523667
Otu000011    0.517000
Otu000057    0.517000
Otu000009    0.515333
Otu000050    0.504333
Otu000047    0.503000
dtype: float64

**-- WHAT IF WE ONLY CONSIDER THOSE OTU's WHICH ARE SIGNIFICANTLY PRESENT IN THE PRODUCTIVITY SAMPLES? --**

In [32]:
'''Import data'''
data_abs = pd.read_csv('data/Chloroplasts_removed/nochloro_absolute_otu.tsv', sep=' ', index_col=None, header=0)
data_rel = pd.read_csv('data/Chloroplasts_removed/nochloro_relative_otu.tsv', sep=' ', index_col=None, header=0)
target = pd.read_csv('data/Chloroplasts_removed/nochloro_HNA_LNA.tsv', sep=' ', index_col=0, header=0)
productivity = pd.read_csv('data/Chloroplasts_removed/productivity_data.tsv', sep=' ', index_col=0, header=0)

In [33]:
'''Set sample names as index and shuffle data'''
data_abs.set_index(target.samples,inplace=True)
data_rel.set_index(target.samples,inplace=True)
data_abs = data_abs.sample(frac=1, random_state=3)
data_rel = data_rel.sample(frac=1, random_state=3)
target = target.sample(frac=1, random_state=3)
productivity = productivity.sample(frac=1, random_state=3)

#Create target columns of HNA-values: 
hna = target.loc[:,'HNA.cells']
hna_rel = hna/target.loc[:,'Total.cells']
hna = pd.Series(hna, index=hna.index)
hna_rel = pd.Series(hna_rel, index=hna_rel.index)

**Preprocessing: ** First filter productivity outliers (productivity > 90). 

In [34]:
#retain only productivity samples 
productivity = productivity.dropna(subset=['tot_bacprod'])
#remove high productivity samples (>90)
productivity = productivity[productivity.tot_bacprod < 90]

idx_prod = productivity.samples.values
#display(idx_prod)
prod = pd.Series(productivity.tot_bacprod.values, index=idx_prod)
#prod_error = pd.Series(productivity.SD_tot_bacprod.values, index=idx_prod)
#prod_rel_error = prod_error/prod

In [35]:
'''Check different abun '''
abun = 0.005

In [36]:
from analysis_functions import preprocess_df
data_abs_prod = data_abs.loc[idx_prod,:] 
data_abs_prod = preprocess_df(data_abs_prod,abun,True)
otus_prod = list(data_abs_prod.columns)

print('Number of OTUs: ' + str(len(otus_prod)))

Number of OTUs: 121


In [37]:
alphas_abs, preds_abs = perform_nested_lasso_cv(data_abs[otus_prod], hna, cv_out, cv_in)
r2_abs = get_r2(hna, preds_abs)
print('R² based on absolute abundances after productivity filtering: ' + str(r2_abs))

R² based on absolute abundances after productivity filtering: 0.852836976091


In [38]:
t=0

for idx_train, idx_test in outer_cv.split(data_abs, hna):
    lassoCV = get_lassoCV(5)
    #scaler = StandardScaler()
    #scaler.fit(data_abs.iloc[idx_train,:])
    #data_abs = pd.DataFrame(scaler.transform(data_abs[otus]),index=data_abs.index,columns=otus)    
    lassoCV.fit(data_abs.ix[idx_train,otus_prod], hna[idx_train])
    mse = np.sum(lassoCV.mse_path_, axis=1)
    mse_min = np.min(mse)
    alpha = lassoCV.alpha_
    
    otu_prod_scores = pd.Series(perform_randomizedLasso(data_abs.ix[idx_train,otus_prod], hna[idx_train], alpha), index=otus_prod)
    otu_scores_cv.loc[t] = otu_prod_scores
    otu_prod_scores.sort_values(ascending=False,inplace=True)
        
    mse_scores = np.zeros(len(thresholds))
    dummy=0
        
    scores = otu_prod_scores
    
    for thr in thresholds: 
        scores = otu_prod_scores[otu_prod_scores.values > thr]
        features_new = scores.index
        if(len(features_new) > 0): 
            lassoCV = get_lassoCV(cv_in)
            lassoCV.fit(data_abs.ix[idx_train,features_new],hna[idx_train])
            #alphas, preds = perform_nested_ridge_cv(data_abs[features_new],hna) #We could use this if we want a different evaluation model
            mse = np.sum(lassoCV.mse_path_, axis=1)
            mse_scores[dummy] = np.min(mse)
        dummy+=1
        
    mse_scores = mse_scores[np.nonzero(mse_scores)]
    #mse_min = mse_scores.min()
    mse_min_idx = mse_scores.argmin()
    thresh_max = thresholds[mse_min_idx]
    thresholds_cv[t] = thresh_max
    optimal_scores = otu_prod_scores[otu_prod_scores.values>thresh_max]
    selected_otus = optimal_scores.index
    
    lassoCV = get_lassoCV(cv_in)
    lassoCV.fit(data_abs.ix[idx_train, selected_otus], hna[idx_train])
    alpha = lassoCV.alpha_
    lasso = Lasso(alpha,max_iter=20000,normalize=True)
    pred_cv_final = cross_val_predict(lasso, data_abs.ix[idx_train, selected_otus], hna[idx_train], cv=cv_in)
    r2_cv[t] = get_r2(pred_cv_final, hna[idx_train])
    pred.iloc[idx_test] = lassoCV.predict(data_abs.ix[idx_test,selected_otus])
    t+=1

In [39]:
r2_final = get_r2(pred,hna)   
print('mean(R²_cv), std(R²_cv): ' + str(r2_cv.mean()) + ', ' +  str(r2_cv.std()))
print('R²_test: ' + str(r2_final))

mean(R²_cv), std(R²_cv): 0.895356559591, 0.0146182009881
R²_test: 0.844020313234


In [40]:
mean_otu_scores = otu_scores_cv.mean()
std_otu_scores = otu_scores_cv.std()
mean_otu_scores.sort_values(ascending=False,inplace=True)
avg_thresh = thresholds_cv.mean()
std_thresh = thresholds_cv.std()
otu_scores_cv.to_csv('HNA_scores_prodfiltering_OTU_stand_10x5CV_abun0.01.csv')
otus_final = mean_otu_scores[mean_otu_scores > (avg_thresh)]#+std_thresh)]
display(otus_final.head(10))

Otu000027    0.962121
Otu000057    0.961515
Otu000005    0.928788
Otu000109    0.908333
Otu000067    0.858485
Otu000123    0.857333
Otu000058    0.855455
Otu000043    0.848333
Otu000050    0.804848
Otu000048    0.675152
dtype: float64

**-- SUMMARY --**

- We are able to predict the changes in the HNA-cloud up to high accuracy $R^2_{HNA(final)} = 0.953$, using our pipeline; This gives us a reduced set of OTU's (340 $\rightarrow$ 100). 
- This score is higher than using a 'standard' Lasso on all OTU's after initial filtering ($R^2_{HNA(abs)} = 0.768$). 
- Our pipeline in which we use absolute abundances gives added value, as only using the total cell counts gives us an $R^2_{HNA(Total.cells)} = 0.712$ 

**-- SUMMARY (Part 2) --**

- If we only include those OTU's which are significantly present in the productivity samples, the results change quite a bit. 
- $R^2_{final} = 0.897$, however the reduced set of OTU's becomes 95 $\rightarrow$ 26.
- This means that we can narrow down our subset of OTU's to 26 if we want to consider those OTU's which are significantly present in the productivity samples and can be related to the HNA-cloud. 